In [ ]:
!pip install googletrans==4.0.0-rc1 gtts -q

from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display
import random
import tempfile
import os
from datetime import datetime
import time

responses = {
    "sad": ["I'm sorry you're feeling down. Remember, tough times don't last.",
            "You are not alone. Talk to me, what's making you sad?",
            "Even storms pass, just like this feeling will.",
            "It's okay to feel sad. Would you like to talk about what's bothering you?"],
    "happy": ["That's awesome! Keep smiling, it suits you.",
              "I love hearing that! What made you so happy today?",
              "Your positive energy is contagious! Tell me more!"],
    "anxious": ["Breathe in... breathe out. You're safe here.",
                "It's okay to feel anxious sometimes. Let's ground together.",
                "Take a moment. What are you anxious about? Let's talk through it."],
    "flirty": ["Oh, someone's in a playful mood!",
               "You're quite the charmer, aren't you?",
               "You've got some moves! But let's keep it friendly."],
    "friendly": ["I'm glad we're chatting! How's your day going?",
                 "Nice talking with you. What's on your mind?",
                 "I'm enjoying our chat! What would you like to talk about?"],
    "love": ["Love makes life brighter. Who's got your heart?",
             "That's sweet! Love is the purest emotion.",
             "Love is beautiful! Tell me about it."],
    "motivational": ["You can do anything if you don't quit.",
                     "Believe in yourself — you've overcome worse before.",
                     "Every small step you take counts. Keep going!"],
    "joke": ["Why don't programmers like nature? It has too many bugs.",
             "What do you call a fake noodle? An impasta!",
             "Why don't scientists trust atoms? Because they make up everything!"],
    "suicidal": ["I'm really sorry you feel this way. You are valuable and loved. Please talk to someone you trust.",
                 "You are not alone. I care about you. Please reach out to a helpline or friend.",
                 "Your life has value. Please call a crisis helpline: they can help you right now."],
    "angry": ["I can sense your frustration. It's okay to feel angry.",
              "Take a deep breath. Let's talk about what's making you upset.",
              "Your feelings are valid. What happened?"],
    "confused": ["It's okay to feel confused. Let's work through this together.",
                 "Take your time. What's confusing you?"],
    "grateful": ["Gratitude is a beautiful feeling! What are you thankful for?",
                 "That's wonderful! Expressing gratitude brings more joy."],
    "default": ["Tell me more about that.", "I see, go on.",
                "Hmm, interesting. How does that make you feel?",
                "I'm listening. Continue...", "That's interesting. What else is on your mind?"]
}

def detect_emotion(text):
    try:
        t = text.lower()
        if any(w in t for w in ["die", "kill myself", "suicide", "end my life", "want to die"]): return "suicidal"
        if any(w in t for w in ["sad", "depressed", "cry", "crying", "lonely", "down", "upset"]): return "sad"
        if any(w in t for w in ["happy", "great", "good", "awesome", "excited", "joy"]): return "happy"
        if any(w in t for w in ["anxious", "nervous", "panic", "worried", "stress", "scared"]): return "anxious"
        if any(w in t for w in ["love", "heart", "crush", "adore", "romantic"]): return "love"
        if any(w in t for w in ["joke", "funny", "laugh", "lol", "haha"]): return "joke"
        if any(w in t for w in ["motivate", "tired", "bored", "lazy", "give up"]): return "motivational"
        if any(w in t for w in ["flirt", "cute", "handsome", "beautiful", "hot"]): return "flirty"
        if any(w in t for w in ["friend", "bro", "buddy", "hello", "hi", "hey"]): return "friendly"
        if any(w in t for w in ["angry", "mad", "furious", "hate", "rage", "annoyed"]): return "angry"
        if any(w in t for w in ["confused", "don't understand", "unclear", "puzzled"]): return "confused"
        if any(w in t for w in ["thank", "grateful", "appreciate", "thanks"]): return "grateful"
        return "default"
    except: return "default"

def speak(text, lang):
    try:
        if not text or not lang: return
        tts = gTTS(text=text, lang=lang, slow=False)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            filename = fp.name
            tts.save(filename)
            display(Audio(filename, autoplay=True))
        time.sleep(0.5)
        try: os.remove(filename)
        except: pass
    except: pass

def translate_text(text, dest_lang, translator):
    if dest_lang == "en" or not text: return text
    for i in range(3):
        try:
            result = translator.translate(text, dest=dest_lang)
            if result and result.text: return result.text
            return text
        except:
            if i < 2: time.sleep(1)
            else: return text
    return text

languages = {
    "1": ("en", "English"), "2": ("ta", "Tamil"), "3": ("hi", "Hindi"),
    "4": ("ja", "Japanese"), "5": ("ko", "Korean"), "6": ("ml", "Malayalam"),
    "7": ("kn", "Kannada"), "8": ("te", "Telugu"), "9": ("es", "Spanish"),
    "10": ("fr", "French"), "11": ("de", "German"), "12": ("zh-cn", "Chinese")
}

def main():
    try:
        print("\n" + "="*60)
        print("MINDMATE - YOUR EMOTIONAL COMPANION")
        print("="*60)
        print("\nChoose your language:")
        for k, v in languages.items(): print(f"{k}. {v[1]}")
        print("="*60)

        choice = input("\nEnter number (default: 1 for English): ").strip()
        if not choice: choice = "1"
        if choice not in languages:
            print("Invalid choice. Using English.")
            choice = "1"

        lang_code, lang_name = languages[choice]
        print(f"\nLanguage set to: {lang_name}\n")

        translator = Translator()
        history = []
        start_time = datetime.now()

        welcome = "Hello! I'm MindMate, here to listen and support you. How are you feeling today?"
        welcome_trans = translate_text(welcome, lang_code, translator)
        print(f"Bot: {welcome_trans}\n")
        speak(welcome_trans, lang_code)

        print("="*60)
        print("INSTRUCTIONS")
        print("="*60)
        print("  Type your message and press Enter to chat")
        print("  Type 'bye', 'exit', or 'quit' to end the session")
        print("  Type 'summary' to see conversation statistics")
        print("="*60 + "\n")

        while True:
            try:
                user_input = input("You: ").strip()
                if not user_input: continue

                user_lower = user_input.lower()

                if user_lower in ["bye", "exit", "quit"]:
                    duration = (datetime.now() - start_time).total_seconds() / 60
                    farewell = "Take care! Remember, you're stronger than you think. Talk to you soon!"
                    farewell_trans = translate_text(farewell, lang_code, translator)
                    print("\n" + "="*60)
                    print("SESSION ENDED")
                    print("="*60)
                    print(f"  Duration: {round(duration, 2)} minutes")
                    print(f"  Total messages: {len(history)}")
                    print("="*60)
                    print(f"\n{farewell_trans}\n")
                    speak(farewell_trans, lang_code)
                    break

                if user_lower == "summary":
                    duration = (datetime.now() - start_time).total_seconds() / 60
                    emotions = {}
                    for h in history:
                        e = h.get('emotion', 'default')
                        emotions[e] = emotions.get(e, 0) + 1
                    print("\n" + "="*60)
                    print("CONVERSATION SUMMARY")
                    print("="*60)
                    print(f"  Duration: {round(duration, 2)} minutes")
                    print(f"  Messages: {len(history)}")
                    if emotions:
                        dominant = max(emotions, key=emotions.get)
                        print(f"  Dominant emotion: {dominant}")
                        print(f"  Emotions detected: {emotions}")
                    print("="*60 + "\n")
                    continue

                emotion = detect_emotion(user_input)
                reply = random.choice(responses.get(emotion, responses["default"]))
                reply_trans = translate_text(reply, lang_code, translator)

                print(f"\nMindMate ({lang_name}): {reply_trans}")
                if emotion != "default": print(f"   [Detected emotion: {emotion}]")
                print()

                speak(reply_trans, lang_code)

                history.append({
                    "user": user_input, "bot": reply,
                    "emotion": emotion, "timestamp": datetime.now().isoformat()
                })
            except KeyboardInterrupt:
                print("\n\nSession interrupted. Type 'bye' to exit.\n")
                continue
            except Exception as e:
                print(f"Error: {str(e)}\n")
                continue
    except: print("\nChatbot terminated.\n")

if __name__ == "__main__":
    main()


THERAPY & EMOTION CHATBOT

Choose your language:
1. English
2. Tamil
3. Hindi
4. Japanese
5. Korean
6. Malayalam
7. Kannada
8. Telugu
9. Spanish
10. French
11. German
12. Chinese

Language set to: English

Bot: Hello! I'm here to listen and support you. How are you feeling today?



INSTRUCTIONS
  Type your message and press Enter to chat
  Type 'bye', 'exit', or 'quit' to end the session
  Type 'summary' to see conversation statistics


Bot (English): Your positive energy is contagious! Tell me more!
   [Detected emotion: happy]

